In [10]:
def get_hw(cur_date=datetime.now().strftime('%Y-%m-%d')):\n",
    "    cur_date = datetime.strptime(cur_date, '%Y-%m-%d')\n",
    "    \n",
    "    if cur_date.weekday()==5:\n",
    "        cur_date = cur_date+timedelta(days=2)\n",
    "    elif cur_date.weekday()==6:\n",
    "        return {'valid': False, 'error': None}\n",
    "    else:\n",
    "        cur_date = cur_date+timedelta(days=1)\n",
    "    \n",
    "    with PoolManager() as http:\n",
    "        \n",
    "        r = http.request('POST', 'https://sh-open.ris61edu.ru/auth/login',\n",
    "                         fields={'login_login': '36_Курдов_Тимофей_36', 'login_password': 'faYUzEUK'})\n",
    "        print(r.status)\n",
    "        if r.status>=400:\n",
    "            return {'valid': False, 'error': r.geturl()[28:]+'->'+str(r.status)}\n",
    "        \n",
    "        cookies = [i for i in re.split('[;,] ', r.getheader('Set-Cookie')) if re.match('(sessionid|NodeID)', i)]\n",
    "        r2 = http.request('GET', 'https://sh-open.ris61edu.ru/personal-area/#diary',\n",
    "                          headers={'Cookie':'; '.join(cookies)})\n",
    "        print(r2.status)\n",
    "        if r2.status>=400:\n",
    "            return {'valid': False, 'error': r2.geturl()[28:]+'->'+str(r2.status)}\n",
    "        \n",
    "        r3 = http.request('POST', 'https://sh-open.ris61edu.ru/api/ScheduleService/GetDiary',\n",
    "                    fields={'date': cur_date.strftime('%Y-%m-%d'), 'is_diary': 'true'},\n",
    "                    headers={'Cookie':'; '.join(cookies),\n",
    "                             'Content-Type': 'application/x-www-form-urlencoded',\n",
    "                             'Content-Length': '29'},\n",
    "                          encode_multipart=False)\n",
    "        print(r3.status)\n",
    "        if r3.status>=400:\n",
    "            return {'valid': False, 'error': r3.geturl()[28:]+'->'+str(r3.status)}\n",
    return {'valid': True, 'content': json.loads(r3.data.decode())['days']}

In [11]:
import re
kw = ['англ', 'алг', 'био', 'гео', 'физр', 'физик', 'лит', 'хим', 'геометр', 'нем', 'фр', 'ист', 'общ', 'рус', 'тех', 'обж', 'родн']
p = re.compile(f".*((что|че).*по.?({'|'.join(kw)})|по.?({'|'.join(kw)}).+(что|че)[- ]?(то)?.*зад.*)", re.IGNORECASE)
groups = p.search('что по родному русскому?').groups()
print(groups)
print(groups[2] if groups[2] else groups[3])

('что по родн', 'что', 'родн', None, None, None)
родн


In [12]:
res = get_hw()

200
200
200


In [13]:
hw = None
if res['valid']:
    for day in res['content'][datetime.now().weekday()+1:-1]:
        for lesson in day['lessons']:
            if 'фаолопр' in lesson['discipline'].lower():
                hw = (lesson['discipline'], day['date'], lesson['homework'])
                break
        if hw:
            break
    if not hw:
        res = get_hw((datetime.now()+timedelta(days=7)).strftime('%Y-%m-%d'))
        if res['valid']:
            for day in res['content'][:datetime.now().weekday()+1]:
                for lesson in day['lessons']:
                    if 'фаолопр' in lesson['discipline'].lower():
                        hw = (lesson['discipline'], day['date'], lesson['homework'])
                        break
                if hw:
                    break
            if not hw:
                print('Ошибка: предмет не найден')
        else:
            print('Ошибка: '+res['error'])
else:
    print('Ошибка: '+res['error'])
print(hw)

200
200
200
Ошибка: предмет не найден
None
